In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.graph_objects as go # data viz

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/coronavirus-data-set-free-api-for-realtime-data/coronavirus.csv


In [10]:
raw_data = pd.read_csv("/kaggle/input/coronavirus-data-set-free-api-for-realtime-data/coronavirus.csv", header=0)
raw_data.head()

,Type,CountryOrRegion,ProvinceOrState,County,Code,Lat,Long,Source,1/22/2020,1/23/2020,...,3/9/2020,3/10/2020,3/11/2020,3/12/2020,3/13/2020,3/14/2020,3/15/2020,3/16/2020,3/17/2020,3/18/2020
0,Confirmed,Afghanistan,NaN,NaN,AF,33.0000,65.0000,NaN,0.0,0.0,...,4.0,5.0,7.0,7.0,7.0,16.0,16.0,21.0,22.0,22.0
1,Deaths,Afghanistan,NaN,NaN,AF,33.0000,65.0000,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Recovered,Afghanistan,NaN,NaN,AF,33.0000,65.0000,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
3,Confirmed,Albania,NaN,NaN,AL,41.1533,20.1683,NaN,0.0,0.0,...,2.0,10.0,12.0,23.0,33.0,38.0,42.0,51.0,55.0,59.0
4,Deaths,Albania,NaN,NaN,AL,41.1533,20.1683,NaN,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0


In [11]:
transformed_data_1 = raw_data[raw_data.columns[8: 62]].fillna(0)
transformed_data_1 = transformed_data_1.astype(int)
transformed_data_1["CountryOrRegion"] = raw_data["CountryOrRegion"]
transformed_data_1["Type"] = raw_data["Type"]
transformed_data_2 = transformed_data_1.groupby(["CountryOrRegion", "Type"]).sum()
transformed_data_2.reset_index(inplace=True) 
transformed_data_2

,CountryOrRegion,Type,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,...,3/6/2020,3/7/2020,3/8/2020,3/9/2020,3/10/2020,3/11/2020,3/12/2020,3/13/2020,3/14/2020,3/15/2020
0,Afghanistan,Confirmed,0,0,0,0,0,0,0,0,...,1,1,4,4,5,7,7,7,16,16
1,Afghanistan,Deaths,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Afghanistan,Recovered,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Albania,Confirmed,0,0,0,0,0,0,0,0,...,0,0,0,2,10,12,23,33,38,42
4,Albania,Deaths,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,Vietnam,Deaths,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
530,Vietnam,Recovered,0,0,0,0,0,0,0,0,...,16,16,16,16,16,16,16,16,16,16
531,Zambia,Confirmed,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
532,Zambia,Deaths,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
transformed_data_2 = transformed_data_2.loc[transformed_data_2["3/15/2020"] >= 100]

confirmed_cases = transformed_data_2.loc[transformed_data_2["Type"] == "Confirmed"]
confirmed_cases.set_index("CountryOrRegion", inplace=True)
confirmed_cases.drop(columns="Type", inplace=True)

deceased = transformed_data_2.loc[transformed_data_2["Type"] == "Deaths"]
deceased.set_index("CountryOrRegion", inplace=True)
deceased.drop(columns="Type", inplace=True)

recovered = transformed_data_2.loc[transformed_data_2["Type"] == "Recovered"]
recovered.set_index("CountryOrRegion", inplace=True)
recovered.drop(columns="Type", inplace=True)

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=confirmed_cases.index,
        y=confirmed_cases["3/15/2020"],
        name="Confirmed",
        marker_color='rgb(55, 83, 109)'
    )
)
fig.add_trace(
    go.Bar(
        x=deceased.index,
        y=deceased["3/15/2020"],
        name="Deaths",
        marker_color='rgb(26, 118, 255)'
    )
)
fig.add_trace(
    go.Bar(
        x=recovered.index,
        y=recovered["3/15/2020"],
        name="Recovered",
        marker_color='rgb(255, 105, 180)'
    )
)
fig.update_layout(title="Count of confirmed cases the 15th of March 2020 (>= 100 cases)",
                 template="seaborn")
fig.show()

In [13]:
case_fatality = deceased["3/15/2020"] / (deceased["3/15/2020"] + recovered["3/15/2020"]) * 100
case_fatality.fillna(0, inplace=True)
case_fatality

CountryOrRegion
China                             4.527160
Cruise Ship: Diamond Princess     0.000000
France                            0.000000
Iran                             13.624388
Italy                            43.653475
Japan                             0.000000
Singapore                         0.000000
South Korea                       0.000000
Spain                            36.093943
Name: 3/15/2020, dtype: float64

In [14]:
fig = go.Figure()
for country in transformed_data_2["CountryOrRegion"].drop_duplicates().tolist():
    fig.add_trace(
        go.Scatter(
            x=confirmed_cases.columns,
            y=confirmed_cases[confirmed_cases.index == country].values[0],
            mode="lines+markers",
            name=country
        )
    )
fig.update_layout(title="Count of confirmed case (cases > 100)")
fig.show()


In [17]:

fig = go.Figure()
for country in confirmed_cases.index:
    confirmed_cases_for_current_country = confirmed_cases[confirmed_cases.index == country].values[0]
    confirmed_cases_for_current_country = [date for date in confirmed_cases_for_current_country if date >= 100]
    fig.add_trace(
        go.Scatter(
            x=np.arange(1, len(confirmed_cases_for_current_country)+1),
            y=confirmed_cases_for_current_country,
            mode="lines+markers",
            #hoverinfo=""
            name=country
        )
    )
fig.update_layout(title="Flattening the curve: day one since the 100th case")
fig.show()

['Australia' 'Confirmed' 0 0 0 0 4 5 5 6 8 9 11 12 12 13 13 14 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 25 27 30 39 51 54
 60 63 76 91 107 128 128 200 250 297]
['Austria' 'Confirmed' 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 2 2 3 3 9 14 18 21 29 41 55 79 104 131 182 246 361 504
 655 860]
['Bahrain' 'Confirmed' 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 23 33 33 36 41 47 49 49 52 55 60 85 85 95 110 195 197 210
 212 214]
['Belgium' 'Confirmed' 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 8 13 23 50 109 169 200 239 267 314 399 559
 689 886]
['Brazil' 'Confirmed' 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 2 2 2 2 4 4 13 13 20 25 31 38 77 151 151 203]
['Canada' 'Confirmed' 0 0 0 0 1 1 2 2 2 4 4 4 4 4 5 5 7 7 7 7 7 7 7 7 7 7
 8 8 8 8 9 9 9 10 11 11 13 14 20 24 27 30 33 37 49 54 64 77 79 108 117 196
 250 339]
['China' 'Confirmed' 548 643 920 140

In [ ]:
confirmed_cases_for_current_country = transformed_data_2.loc[transformed_data_2["CountryOrRegion"] == "France"]
confirmed_cases_for_current_country